# WeLoveDogs Twitter data analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#assessing">Assessing Data</a></li>
<li><a href="#cleaning">Data Cleaning</a></li>
<li><a href="#storing">Storing Data</a></li>
<li><a href="#analysis">Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
# Introduction

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import os
import timeit
from datetime import datetime
import re

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Setting up Twitter API:

In [ ]:
import tweepy

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

<a id='wrangling'></a>
# Data Wrangling 

## WeRateDogs Twitter archive

In [ ]:
df = pd.read_csv('twitter-archive-enhanced.csv', dtype= {'tweet_id': 'str'})

In [ ]:
df.shape

In [ ]:
df.head(3)

## Tweet image predictions

In [ ]:
folder_name = 'images_pred'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
url = r'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)

In [ ]:
os.listdir(folder_name)

In [ ]:
df_pred = pd.read_csv(r'images_pred/image-predictions.tsv', sep='\t')

In [ ]:
df_pred.shape

In [ ]:
df_pred.head(5)

## Tweets' retweet count and favorite ("like") count (at minimum)

To identify the metadata in the API call I used: https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2

In [ ]:
# List of dictionaries to build and convert to a DataFrame later
data_tweet = {}
data_tweet['tweet'] = []
ranking = 0
start = timeit.timeit()
for tweetid in df.tweet_id:
    try:
        tweet = api.get_status(tweetid, tweet_mode = 'extended', wait_on_rate_limit= True, wait_on_rate_limit_notify = True)
        retweets = str(tweet.retweet_count)
        likes = str(tweet.favorite_count)
        data_tweet['tweet'].append({'tweet_id': tweetid,
                                    'retweet': retweets,
                                    'likes': likes})
    
    except Exception as e:
        print(str(ranking) + "_" + tweetid + ": " + str(e))
        
end = timeit.timeit()
print(end - start)

A few tweet ids failed to retrieve data, but the above code run as normal. Now writting the content as JSON in a .txt file:

In [ ]:
with open('tweet_json.txt', 'w') as outfile:
    json.dump(data_tweet, outfile)

One extra step is needed here: to use the JSON library to transform the json-format content in a dataframe-friendly format

In [ ]:
import json

with open('tweet_json.txt') as obj:
    data = json.load(obj)
    print(type(data))
    df_pop = pd.DataFrame.from_dict(data['tweet'])

In [ ]:
with open('tweet_json.txt') as json_file:
    data = json.load(json_file)
    print(len(data['tweet']))

In [ ]:
df.tweet_id.nunique()

In [ ]:
#Check what is missing
miss_tweet = []
with open('tweet_json.txt') as json_file:
    data = json.load(json_file)
    for p in data['tweet']:
        miss_tweet.append(p['tweet_id'])    

In [ ]:
len(miss_tweet)

In [ ]:
print(set(df.tweet_id) - set(miss_tweet))

In [ ]:
df.columns

In [ ]:
url = df[df['tweet_id'] == '837012587749474308'].expanded_urls.astype(str)
print(url)

Could not find those tweets manually on Twitter.com, it seems they have been deleted or the tweet id is faulty. 
We should remove those tweets from the datasets

<a id='assessing'></a>
# Assessing Data

3 dataframes:

- `archive` the archive of tweets
- `prediction`, image prediction algo results
- `likes and count of retweets`, downloaded number of retweets and likes

## Archive of the tweets

### Visual Assessment

In [ ]:
df.head()

In [ ]:
df.in_reply_to_user_id.unique()

### Programmatic Assessment

In [ ]:
df.info()

In [ ]:
print(type(df.tweet_id[0]))

In [ ]:
df[df['retweeted_status_id'].notnull()]

The columns 'retweeted_status_id' and 'retweeted_status_timestamp' don't bring much value to the analysis in the current context

In [ ]:
df.query('rating_denominator != "10"').sample(5)

- 'rating_numerator' has values < 10 (not the logic) => after manual check noticed that it may be either because the numerator has been scrapped from a number in the text that is not the rating or the user didn't follow the rating system underlying logic. As a result, to maintain consistency and make the tweets comparable, drop the lanies with numerator < 10
- 'rating_denominator' has values different from 10 => same as above, drop the lines with values different from 10

Columns 'in_reply_to_status_id' and 'in_reply_to_user_id' have only 78 values
Timestamp not easy to read
Text is raw
Source is HTML format
'retweeted_status_id' and 'retweeted_status_user_id' not actionable
Doggo/Fluffer/Puppo etc => same variable, should be in the same column
'rating_numerator' has values < 10 (not the logic)
'denominator has various values when it should be 10 for consistency (text parsing issue)

In [ ]:
df.columns

In [ ]:
df_pred.columns

In [ ]:
df_pop.columns

In [ ]:
df.expanded_urls.value_counts()

In [ ]:
df[df['expanded_urls'] == 'https://twitter.com/dog_rates/status/667866724293877760/photo/1']

In [ ]:
df[df['expanded_urls'] == 'https://twitter.com/dog_rates/status/698195409219559425/photo/1']

In [ ]:
df[df.expanded_urls.duplicated()]

It looks like there are some duplicates generated by retweets and comments in the dataframe


In [ ]:
df[df.text.str.match(r'^RT')]

In [ ]:
df.expanded_urls.value_counts()

In [ ]:
df[df['expanded_urls'] == 'https://www.gofundme.com/my-puppys-double-cataract-surgery,https://twitter.com/dog_rates/status/825026590719483904/photo/1,https://twitter.com/dog_rates/status/825026590719483904/photo/1']

Some Gofundme links for people raising money for their dogs surgery. Some commercial links, often because of partnerships with the Twitter account.

In [ ]:
df.name.value_counts()

Many dogs' names are not populated, a lost of 'a' and a few 'the' which doesn't sounds like a dog's name

##  Image prediction dataframe

### Visual Assessment

In [ ]:
df_pred.head(5)

If the first prediction is not a dog it may be that the second or third one is. Although, the confidence is very low in that case. To maintain reliability, we should probably drop the second and third suggestions. Then, we should clean the first suggestion.

### Programmatic Assessment

In [ ]:
df_pred.query('p1_dog == False')

In [ ]:
df_pred.p2_dog.unique()

Columns p1, p2, p3 have values which are not dog breeds


In [ ]:
df_pred.jpg_url.value_counts()

In [ ]:
df_pred[df_pred.jpg_url == 'https://pbs.twimg.com/media/CrXhIqBW8AA6Bse.jpg']

DUplicates due to retweets

## Retweets and likes dataframe

### Visual Assessment

In [ ]:
df_pop.head(5)

### Programmatic Assessment

In [ ]:
df_pop.info()

In [ ]:
df_pop.describe()

## Data Assessment Conclusion

### Quality issues:

##### `archive` table
- Columns 'in_reply_to_status_id' and 'in_reply_to_user_id' have only 78 values. Columns 'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp' only have 181
- Timestamp column not easy to deal with
- Duplicates generated by retweets and comments
- Many dogs' names are not populated, a lost of 'a' which doesn't sounds like a dog's name
- Source is HTML format
- 'rating_numerator' has values < 10 (not the logic)
- 'rating_denominator' has values different from 10 (text parsing issue)

##### `prediction` table
- Columns p1, p2, p3 have values which are not dog breeds

### Tidiness issues:

##### `archive` table
- df archive: The columns Doggo/Fluffer/Puppo are the same variable and should be consolidated in the same column

##### `retweets and likes` table
- Likes and count of retweet are attributes of the archived tweets

<a id='cleaning'></a>
# Data Cleaning

In [ ]:
df_clean = df.copy()
df_pred_clean = df_pred.copy()
df_pop_clean = df_pop.copy()

## Tidiness

#### `archive` table: The columns Doggo/Fluffer/Puppo are the same variable and should be consolidated in the same column

##### Define
Consolidate the different values from the column into on new column (the variable)

In [ ]:
df_clean.columns

##### Code

In [ ]:
df_clean = pd.melt(df_clean, id_vars=['tweet_id', 
                                      'in_reply_to_status_id',
                                      'in_reply_to_user_id',
                                      'timestamp', 
                                      'source', 
                                      'text', 
                                      'retweeted_status_id', 
                                      'retweeted_status_user_id',
                                      'retweeted_status_timestamp',
                                      'expanded_urls',
                                      'rating_numerator', 
                                      'rating_denominator', 
                                      'name'],
                                       var_name='dog_type')
df_clean.drop(columns='dog_type', inplace=True)
df_clean.rename(columns={'value' : 'dog_type'}, inplace=True)

##### Test

In [ ]:
df_clean.dog_type.unique()

#### `retweets and likes` table: likes and count of retweet are attributes of the archived tweets

##### Define
Merge the retweets and likes dataframe with the archive dataframe. Then, fill the NaN values generated by the discrepancies between the two

##### Code

In [ ]:
df_pop.tweet_id = df_pop.tweet_id.astype(str)
df_clean = pd.merge(df_clean, df_pop,
                            on=['tweet_id'], how='left')

In [ ]:
df_clean.head(3)

In [ ]:
type(df_clean.likes[0])

Likes and retweets have been marged as float. Moreover, some tweets don't have likes/retweets values to be attributted, resulting in NaN values. 
We need to fill in the missing values with the median to avoid skewing the data analysis.Then we will be able to convert the two columns into a integer data type.

In [ ]:
#Filling NaN with median value
df_clean.likes.fillna(df_clean.likes.median(), inplace=True)
df_clean.retweet.fillna(df_clean.retweet.median(), inplace=True)

In [ ]:
#Converting the likes and counts of retweet into integer
df_clean.likes = df_clean.likes.astype(int)
df_clean.retweet = df_clean.retweet.astype(int)

##### Test

In [ ]:
df_clean.info()

## Quality

#### `archive` table: Columns 'in_reply_to_status_id' and 'in_reply_to_user_id' have only 78 values. Columns 'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp' only have 181

##### Define
Drop the columns 

##### Code

In [ ]:
df_clean.columns

In [ ]:
df_clean.drop(columns=['in_reply_to_status_id', 
                       'in_reply_to_user_id', 
                       'retweeted_status_id', 
                       'retweeted_status_user_id', 
                       'retweeted_status_timestamp'], inplace=True)

##### Test

In [ ]:
df_clean.columns

#### `archive` table: Source is HTML format

##### Define
Create a dictionnary with the different sources and more simple names. Apply a method to replace with the simple names.

In [ ]:
df_clean.source.unique()

##### Code

In [ ]:
source = {'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>' : 'Iphone',
          '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>' : 'Web browser',
         '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>' : 'Vine',
         '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>' : 'TweetDeck'
         }

df_clean.replace(source, inplace=True)

##### Test

In [ ]:
df_clean.source.unique()

#### `archive`  Timestamp column not easy to deal with

#### Define
Use timedate to convert to date, drop the time since it doesn't really bring much value to our analysis

##### Code

In [ ]:
df_clean.timestamp = df_clean.timestamp.astype(str).str[:-15]
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

##### Test

In [ ]:
type(df_clean.timestamp[0])

In [ ]:
df_clean.head(2)

#### `archive` table: Duplicates generated by retweets and comments

##### Define
Drop the row generated with retweets and replies using regular expressions

##### Code

In [ ]:
df_clean = df_clean[df_clean.text.str.contains(r'^(?!RT)')]
df_clean = df_clean[df_clean.text.str.contains(r'^(?!@)')]

##### Test

In [ ]:
print(df_clean[df_clean.text.str.contains(r'^RT')])
print(df_clean[df_clean.text.str.contains(r'^@')])

#### `archive` table: Many dogs' names are not populated, a lost of 'a' which doesn't sounds like a dog's name

##### Define
Replace the odd names with NaN values

##### Code

In [ ]:
df_clean.loc[df_clean.name.str.islower(), 'name'] = np.nan

##### Test

In [ ]:
df_clean.name.value_counts()

#### `archive` table: Numerator column has values lower than 10 (not the logic)

##### Define
Filter for removing the rows with numerator lower than 10

##### Code

In [ ]:
df_clean = df_clean.query('rating_numerator >= 10')

##### Test

In [ ]:
print(df_clean.rating_numerator.unique())

#### `archive` table:Denominator column has values different from 10 

##### Define
Set the value of the entire column to 10

##### Code

In [ ]:
df_clean['rating_denominator'] = 10

##### Test

In [ ]:
print(df.rating_denominator.unique())

## Tweet image predictions

#### `image prediction` table: The columns p1, p2 and p3 have values which are not dog breeds

#### Define
Drop the columns for the second and third suggestions as their associated prediction likelihood is very low. Filter for True dogs suggestions to clean the names

##### Code

In [ ]:
#Dropping the columns
df_pred_clean.drop(df_pred_clean.columns[6:], axis=1, inplace=True)

In [ ]:
#Filter for True dog suggestions
df_pred_clean = df_pred.query('p1_dog == True')

##### Test

In [ ]:
df_pred_clean.columns

In [ ]:
df_pred_clean.p1.value_counts()

<a id='storing'></a>
# Storing Data

We are storing the main dataframe into twitter_archive_master.csv

An additional file is required for tidiness. Thus, we are storing the results of the images prediction algorithm into image_prediction.csv

In [ ]:
df_clean.to_csv('twitter_archive_master.csv', index=False)
df_pred_clean.to_csv('image_prediction.csv', index=False)

<a id='analysis'></a>
# Data Analysis

## Questions to answer

The analysis will be broken down in 3 parts as follow:
- **Dogs population insights**
- **'Owners' popularity** (The grades given by the dog owners directly on the post. Very important but maybe not the most objective metric to look at...?)
- **'Public' popularity** (Popularity in terms of likes and retweets, attributed by the public on Twitter. An interesting approach to understand who are the most famous dogs around)

Therefore, the questions to answer are:

**Which source are the dog lovers using to share their best pictures?** 
<br>**Which type of dogs gets the best grades 'out of 10'?**
<br>**Which breeds of dogs are the most popular in terms of likes and retweets?**
<br>**Has a type of dog been more popular than the others over time?**
<br>**Are the types of dogs with good grades also popular in on Twitter?**

In [ ]:
#reloading the data from the CSV files
df_clean = pd.read_csv('twitter_archive_master.csv', dtype={'tweet_id':'str'})
df_pred_clean = pd.read_csv('image_prediction.csv', dtype={'tweet_id':'str'})

In [ ]:
df_clean.head(2)

In [ ]:
df_pred_clean.head(2)

### About the dogs population

In [ ]:
#To be able to look specificaly at the dog type related metrics, let's create a dataframe with only the observations populated with a specific type
df_clean_nona = df_clean.query('dog_type != "None"')

In [ ]:
order = df_clean_nona.dog_type.value_counts().index
hue = sns.color_palette()[2]
sns.countplot(y='dog_type', data= df_clean_nona, order=order, color=hue)
plt.ylabel('Dog type')
plt.xlabel('')
plt.title('Population size per dog type');

The pupper represent the most important poopulation of dogs with a population of nearly 200. The floofers are the smallest population.

### About the owner popularity

In [ ]:
df_clean.groupby('dog_type').mean()

The puppos are receiving the best grades 'out of 10' on average, with an average of 12.2/10

### About the public popularity

Checking what the likes and retweets distributions look like:

In [ ]:
df_clean.likes.hist(figsize=(12, 6), bins=50);

In [ ]:
df_clean.retweet.hist(figsize=(12, 6), bins=50);

Both distributions are very skewed on the left.

In [ ]:
#sum the likes per dog type?
df_clean.groupby('dog_type').sum()

The puppers are cumulating the most likes and retweets, but are twice as many as the doggos (dividing the denominator by 10 to get the number).

In [ ]:
df_clean.groupby('dog_type').describe()

Despite being a small portion of the total population, the puppos have an higher average number of likes around 22k.

In [ ]:
df_clean.groupby('source').mean()

People posting from an Iphone are on average receiving more likes. Although, The Vine-made and web browser-originated tweets are more retweeted.

To be able to develop the study to get further insights about the dog breeds, we need to merge the two dataframes on the tweet_id key:

In [ ]:
master_df = pd.merge(df_clean, df_pred_clean, on=['tweet_id'], how='left')

In [ ]:
master_df.info()

In [ ]:
master_df.head(5)

Let's now group the new dataframe by dog breed and check the who is at the top of the rankings in terms of likes and retweets:

In [ ]:
master_df.groupby('p1').mean().sort_values('likes', ascending=False).head(5)

In [ ]:
master_df.groupby('p1').mean().sort_values('retweet', ascending=False).head(5)

The black and Tan Coonhound and Standard Poodle are the most popular breed in terms of number of likes and number of retweets. They both rank first and second in those two categories.

To get further insight about the dogs popularity per type, let's add another feature which is the distribution of the type's population in terms of likes:

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = sns.stripplot(y='likes', x=df_clean_nona['dog_type'], data=df_clean_nona, jitter=True, dodge=True, alpha=0.5)
ax.set_title('Distribution of the number of likes for each dog type',fontsize=18)
ax.set_xlabel('Dog type',fontsize=18)
ax.set_ylabel('Number of likes',fontsize=18)
ax.yaxis.grid(color='white')
ax.xaxis.grid(color='white')

We can see that of few doggos have received a landslide appraisal, as for a pupper and a puppo. But most of their peers have an average of 10k likes. The floofers are more rare than the other dog types. The most populated type is the pupper type.

Finally, I would like to understand if the dog's owners and the public get along when it comes to rating those dogs. To find out, let's categorize the dogs in terms of owner's popularity and compare each category's public approval:

In [ ]:
df_clean.describe()

In [ ]:
#Popularity per mark
bin_edges = [ 0, 10, 11, 12, 1776]
bin_names = [ 'Low', 'Medium', 'Moderate', 'High']
df_clean['popularity'] = pd.cut(df_clean['rating_numerator'], bin_edges, labels=bin_names)
df_clean.head(5)

In [ ]:
category_df = df_clean.groupby('popularity').mean()
category_df.reset_index(level=0, inplace=True)
category_df

In [ ]:
order_pop = category_df.sort_values('likes', ascending=False).iloc[:,0]

sns.barplot(x='likes',y='popularity', data=category_df, order=order_pop, color= hue)
plt.xlabel('Average number of likes')
plt.ylabel('Popularity category')
plt.title('Average number of likes (public popularity) per category of dog rate (owner popularity)');

The upper quartile which represents the 25% of posts with the best grade are proven to be also the most popular among the users, receiving the highest average number of likes and retweets.

<a id='conclusions'></a>
# Conclusions

- **Dogs population insights**

Most of the tweets are sharing puppers.

- **'Owners' popularity** 

The puppos are on average receiving the best grades from their owners with an average of 12.2/10.

- **'Public' popularity** 

The public appreciate the most the dogs' pictures posted from an Iphone, but share rather the post made with Vine and from a web browser.
Among the breeds, the black and Tan Coonhound and the Standard Poodle are the most popular dogs for the Twitter users. The distribution of tweets popularity metrics are displaying a few extremely popular posts but overwhelmingly the dogs received a rather stable and low number of likes and retweets.
Finally, there is a clear correlation between the owner's rates and the public's popularity: the quartile of dogs which received the highest owner's grades are also receiving the most likes on average from the dog lovers spending time on WeRateDogs.